INFO:     Started server process [53311]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enab

INFO:     127.0.0.1:51830 - "POST /press HTTP/1.1" 200 OK
INFO:     127.0.0.1:51830 - "POST /press HTTP/1.1" 200 OK
INFO:     127.0.0.1:51830 - "POST /press HTTP/1.1" 200 OK
INFO:     127.0.0.1:51830 - "POST /press HTTP/1.1" 200 OK
INFO:     127.0.0.1:51830 - "POST /press HTTP/1.1" 200 OK
INFO:     127.0.0.1:51830 - "POST /press HTTP/1.1" 200 OK
INFO:     127.0.0.1:51830 - "POST /press HTTP/1.1" 200 OK
INFO:     127.0.0.1:51830 - "POST /press HTTP/1.1" 200 OK
INFO:     127.0.0.1:51830 - "POST /press HTTP/1.1" 200 OK
INFO:     127.0.0.1:51830 - "POST /press HTTP/1.1" 200 OK
INFO:     127.0.0.1:51830 - "POST /press HTTP/1.1" 200 OK
INFO:     127.0.0.1:51830 - "POST /press HTTP/1.1" 200 OK
INFO:     127.0.0.1:51830 - "POST /press HTTP/1.1" 200 OK
INFO:     127.0.0.1:51830 - "POST /press HTTP/1.1" 200 OK
INFO:     127.0.0.1:51830 - "POST /press HTTP/1.1" 200 OK
INFO:     127.0.0.1:51830 - "POST /press HTTP/1.1" 200 OK
INFO:     127.0.0.1:51830 - "POST /release HTTP/1.1" 200 OK
INFO:     12

In [2]:
from isaacsim import SimulationApp

# Set the path below to your desired nucleus server
# Make sure you installed a local nucleus server before this
simulation_app = SimulationApp()

[Warning] [simulation_app.simulation_app] fast shutdown not supported with jupyter notebooks
Starting kit application with the following args:  ['/home/ronim/isaacsim/_build/linux-x86_64/release/exts/isaacsim.simulation_app/isaacsim/simulation_app/simulation_app.py', '/home/ronim/isaacsim/_build/linux-x86_64/release/apps/isaacsim.exp.base.python.kit', '--/app/tokens/exe-path=/home/ronim/isaacsim/_build/linux-x86_64/release/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True', '--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--/app/fastShutdown=False', '--/app/installSignalHandlers=0', '--ext-folder', '/home/ronim/isaacsim/_build/linux-x86_64/release/exts', '--ext-folder', '/home/ronim/isaacsim/_build/linux-x86_64/release/apps', '--/physics/cudaDe

[12.711s] Simulation App Startup Complete


In [ ]:
from ServerManager import ServerManager
import subprocess


WIDTH = 640
HEIGHT = 480
RENDERING_FREQUENCY = 30  # Frames per second

# ServerUtils.start_fastapi()
ffmpeg_process = subprocess.Popen([
    'ffmpeg',
    # Input Flags
    '-f', 'rawvideo',
    '-pix_fmt', 'rgb24',        # The pixel format of the data from Numpy
    '-s', f'{WIDTH}x{HEIGHT}',  # Input resolution
    '-r', str(RENDERING_FREQUENCY), # Match the FFmpeg input rate to our render frequency
    '-i', '-',                  # Input from stdin pipe

    # Output Flags
    '-c:v', 'libx264',          # Codec
    '-preset', 'ultrafast',     # Encoding speed vs. compression
    '-tune', 'zerolatency',     # CRITICAL: Optimizes for real-time streaming
    '-pix_fmt', 'yuv420p',      # IMPORTANT: Most web players require this pixel format.
    '-bufsize', '1000k',        # Video buffer size
    '-g', str(RENDERING_FREQUENCY * 2), # Group of Pictures (GOP) size (e.g., a keyframe every 2 seconds)
    '-f', 'mpegts',             # Muxer format for streaming
    '-'                         # Output to stdout pipe
], stdin=subprocess.PIPE, stdout=subprocess.PIPE)


server = ServerManager(ffmpeg_process)





# Start background threads
server_thread = threading.Thread(target=server.start(), daemon=True)
server_thread.start()

encoding_thread = threading.Thread(target=server.encode_frames_from_queue(), daemon=True)
encoding_thread.start()


In [ ]:
from isaacsim.core.api import World
from isaacsim.core.api.objects import DynamicCuboid
import numpy as np

import ServerUtils
from  AssetClass import AssetClass
from CameraClass import CameraClass
from StageClass import StageClass
from PhysicsClass import PhysicsClass

dynamic_cube = DynamicCuboid(prim_path="/World/cube")

cube = AssetClass(dynamic_cube)

cube.update_prim_transformation(translation=np.array([5, 0, 0]), orientation = None)
camera = CameraClass(prim_path = "/World/camera",orientation = np.array([0, 90, 0]))



In [3]:
world = World()
world.scene.add_default_ground_plane()



In [ ]:

world.scene.remove_object("dynamic_cube")

world.scene.add(cube.visual)
stage = StageClass()
stage.add_dome_light(light_path="/World/dome_light",intensity= 1500)

world.reset()
camera.camera.initialize()


physics = PhysicsClass(speed=5)


In [ ]:
while simulation_app.is_running():

    velocity = physics.update_velocity_direction(server.input_manager.get_pressed_keys())
    
    cube.rigid.set_linear_velocities(velocity)
    
    world.step(render=True)

    frame_rgba = camera.camera.get_rgba()


    frame_rgb_bytes = frame_rgba[:, :, :3].astype(np.uint8).tobytes()
    server.add_frame_to_queue(frame_bytes).put(frame_rgb_bytes)  # Add the frame to the queue for processing

    camera.camera.capture_frame()
    camera.camera.encode_frames_from_queue()
